### Imports

In [2]:
import os, sys
import pandas as pd
import numpy as np
#pip install msgpack
!{sys.executable} -m pip install --user msgpack
!{sys.executable} -m pip install --user yfinance
!{sys.executable} -m pip install --user tensorflow==2.2

import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import random

import tensorflow
from tensorflow.keras import Input, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

from time import time
from tensorflow.python.keras.callbacks import TensorBoard

from indicators import *
from data_loading import *
from util_functions import *

!{sys.executable} -m pip install --user shap
import shap


physical_devices = tensorflow.config.list_physical_devices(device_type = 'None')
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


### Parameters

In [3]:
trail_size=29
predict_length = 5
time_period = '6y'

### Load data

In [4]:
top100tech = ['AAPL', 'MSFT', 'TSM', 'NVDA', 'INTC', 'ASML', 'ADBE', 'CRM', 'AVGO', 'ORCL', 'CSCO', 'ACN', 'TXN', 'SHOP', 'QCOM', 'SAP', 'SNE', 'AMAT', 'INTU', 'NOW', 'SQ', 'IBM', 'MU', 'AMD',
'UBER', 'FIS', 'LRCX', 'FISV', 'SNOW', 'INFY', 'TEAM', 'ADSK', 'DELL', 'WDAY', 'VMW', 'ADI', 'NXPI', 'KLAC',
'CRWD', 'PLTR', 'DOCU', 'TEL', 'ERIC', 'MCHP', 'CDNS', 'CTSH', 'APH', 'SNPS', 'HPQ',
'PANW', 'OKTA','RNG','STM','MRVL','XLNX','WIT','MSI','SWKS','GLW','DDOG','ANSS','U','ZS','FTNT','STNE','ZBRA','KEYS','FTV','MXIM','COUP','GRMN','SPLK','HUBS','WORK','NET','FLT','CDW','CAJ','AFRM','PAYC','TER',
'VRSN','WDC', 'ZI', 'UMC','ANET','EPAM', 'UI','XM',
'GIB','TRMB', 'TYL','WIX','HPE','LYFT', 'LOGI', 'MPWR']
#This is only 97 stocks rather than 100
top10tech = top100tech[:10]

In [5]:
top_energy = ["XOM", "CVX", "RDS-A", "RDS-B", "PTR", "TOT", "BP",
"SNP", "ENB", "COP", "EQNR", "PBR-A", "PBR", "EPD",
"TRP", "E", "EOG", "SLB", "KMI", "PSX", "CNQ",
"MPC", "PXD", "SU", "VLO"]

In [6]:
data = get_data_yf(top_energy, time_period)

[*********************100%***********************]  25 of 25 completed


In [7]:
data.isna().sum().sum()

0

In [8]:
data = process_stock_data_from_yf(data)

In [9]:
has_data_ratio, data = filter_has_all_data(data)
has_data_ratio

100.0

### Compute technical indicators

In [10]:
# compute all our technical markers for each of the stocks. 
dataset = data.groupby('Ticker').apply(lambda x: add_technical_markers(x, trail_size))

### Choose features

In [11]:
features = ['Close', 'High','Low', 'Open', 'Volume','HH', 'LL', 'AV','SMA', 'SD', 'WILLR','ATR', 'DMH', 'DML','EMA', 'WMA', 'BBHIGH','BBLOW', 'PERBHIGH', 'PERBLOW','TRIMA', 'RSI', 'DX','PDI', 'NDI', 'ADX','ROC', 'MACD', 'CCI']

#correlation

#features = ['Close', 'High','Low', 'Open','HH', 'LL','SMA','ATR','EMA', 'WMA', 'BBHIGH','BBLOW','TRIMA']

#PCA
#features = ['Close', 'High','Low', 'Open', 'Volume']
 
 
 #PCA correlation crossref
#features = ['Close', 'High','Low', 'Open', 'Volume','HH', 'LL','SMA','ATR', 'DMH', 'DML','EMA', 'WMA', 'BBHIGH','BBLOW','TRIMA']
 

dataset = dataset[['Ticker', 'Date'] + features]
num_features = len(features)

### Normalization
We want to normalize the prices, and there are two challenges
1. We want to be able to recover performance metrics like RMSE in dollars, so we have to have the saved transforms for closing price to un transform after prediction. 
2. We will want to normalize the data within each year, because the stock will tend to have different averages in different years
3. We will want to normalize each different company differently, because the stocks will be of differing magnitudes

### Train test split

In [12]:
train = dataset.loc[dataset['Date'] < "2018-03-01"]
valid = dataset.loc[(dataset['Date'] > "2018-03-13") & (dataset['Date'] < "2019-03-01")]
test = dataset.loc[(dataset['Date'] > "2019-03-01") & (dataset['Date'] < "2020-03-01")]

#### Per stock normalization
We make sure to fit the transforms on the training, and
apply them on training, validation, and test

In [13]:
scaler_models = norm_per_stock_split(train, valid, test, features, MinMaxScaler)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

#### Prepare training inputs and outputs
Convert the daily data points into a set of data points each with {trail_size} days.   
We keep track of the tickers so we can later inverse the normalization with the right model. 

In [14]:
x_train, y_train, tickers_train = roll_all_stocks(train, trail_size, predict_length)
x_valid, y_valid, tickers_valid = roll_all_stocks(valid, trail_size, predict_length)
x_test, y_test, tickers_test = roll_all_stocks(test, trail_size, predict_length)

#### Confirm shapes

In [15]:
print("x_train: {}".format(x_train.shape))
print("y_train: {}".format(y_train.shape))
print("x_valid: {}".format(x_valid.shape))
print("y_valid {}".format(y_valid.shape))
print("x_test: {}".format(x_test.shape))
print("y_test {}".format(y_test.shape))

x_train: (16850, 29, 29)
y_train: (16850,)
x_valid: (5225, 29, 29)
y_valid (5225,)
x_test: (5450, 29, 29)
y_test (5450,)


In [16]:
print("tickers_train: {}".format(tickers_train.shape))
print("tickers_train: {}".format(tickers_valid.shape))
print("tickers_train: {}".format(tickers_test.shape))

tickers_train: (16850,)
tickers_train: (5225,)
tickers_train: (5450,)


### Model design

In [17]:
model = Sequential()
model.add(LSTM(units=75, input_shape=(trail_size, num_features), return_sequences=True))
model.add(Dropout(0.25))
model.add(LSTM(units=30, return_sequences=True))
model.add(Dropout(0.25))

model.add(LSTM(units=30, return_sequences=True))
model.add(Dropout(0.25))

model.add(Flatten())
# model.add(Dense(50, activation='relu'))
# model.add(Dense(25, activation='relu'))
model.add(Dense(1))

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 29, 75)            31500     
_________________________________________________________________
dropout (Dropout)            (None, 29, 75)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 29, 30)            12720     
_________________________________________________________________
dropout_1 (Dropout)          (None, 29, 30)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 29, 30)            7320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 29, 30)            0         
_________________________________________________________________
flatten (Flatten)            (None, 870)               0

In [19]:
time_stamp = time()
tensorboard = TensorBoard(log_dir='logs/{}'.format(time_stamp))

In [20]:
model.compile(optimizer='adam', loss='mean_squared_error')

### Train on many stocks

In [ ]:
history1 = model.fit(x_train, y_train, epochs=45, validation_data=(x_valid, y_valid), batch_size=64, shuffle=False, callbacks=[tensorboard])

Epoch 1/45
264/264 [==============================] - 9s 35ms/step - loss: 0.0202 - val_loss: 0.1015
Epoch 2/45
264/264 [==============================] - 9s 34ms/step - loss: 0.0150 - val_loss: 0.0733
Epoch 3/45
264/264 [==============================] - 12s 47ms/step - loss: 0.0173 - val_loss: 0.0499
Epoch 4/45
264/264 [==============================] - 11s 42ms/step - loss: 0.0135 - val_loss: 0.0449
Epoch 5/45
264/264 [==============================] - 12s 45ms/step - loss: 0.0111 - val_loss: 0.0426
Epoch 6/45
264/264 [==============================] - 11s 43ms/step - loss: 0.0098 - val_loss: 0.0434
Epoch 7/45
264/264 [==============================] - 12s 44ms/step - loss: 0.0088 - val_loss: 0.0429
Epoch 8/45
264/264 [==============================] - 11s 41ms/step - loss: 0.0079 - val_loss: 0.0447
Epoch 9/45
264/264 [==============================] - 12s 45ms/step - loss: 0.0077 - val_loss: 0.0429
Epoch 10/45
264/264 [==============================] - 12s 45ms/step - loss: 0.0073 

### Prepare data for one stock

In [ ]:
target_stock = 'XOM'

In [ ]:
single_stock_dataset = dataset[dataset['Ticker']==target_stock].reset_index(drop=True)

In [ ]:
sns.lineplot(x=single_stock_dataset['Date'], y=single_stock_dataset['Close'])

### Train test split

In [ ]:
train = single_stock_dataset.loc[single_stock_dataset['Date'] < "2018-03-01"]
valid = single_stock_dataset.loc[(single_stock_dataset['Date'] > "2018-03-13") & (single_stock_dataset['Date'] < "2019-03-01")]
test = single_stock_dataset.loc[(single_stock_dataset['Date'] > "2019-03-01") & (single_stock_dataset['Date'] < "2020-03-01")]

In [ ]:
single_stock_scaler_models = norm_per_stock_split(train, valid, test, features, MinMaxScaler)

#### Prepare training inputs and outputs
Since we are using just one stock, we don't need to keep track of tickers

In [ ]:
x_train, y_train, _ = roll_all_stocks(train, trail_size, predict_length)
x_valid, y_valid, _ = roll_all_stocks(valid, trail_size, predict_length)
x_test, y_test, _ = roll_all_stocks(test, trail_size, predict_length)

#### Confirm shapes

In [ ]:
print("x_train: {}".format(x_train.shape))
print("y_train: {}".format(y_train.shape))
print("x_valid: {}".format(x_valid.shape))
print("y_valid {}".format(y_valid.shape))
print("x_test: {}".format(x_test.shape))
print("y_test {}".format(y_test.shape))

### Test model on single stock

In [ ]:
print("Training RMSE = {}".format(evaluate_model_rmse(model.predict(x_train), 
                                                      y_train,
                                                     num_features,
                                                     single_stock_scaler_models[target_stock])))

In [ ]:
print("Validation RMSE = {}".format(evaluate_model_rmse(model.predict(x_valid), 
                                                      y_valid,
                                                     num_features,
                                                     single_stock_scaler_models[target_stock])))

In [ ]:
print("Testing RMSE = {}".format(evaluate_model_rmse(model.predict(x_test), 
                                                     y_test,
                                                    num_features,
                                                    single_stock_scaler_models[target_stock])))

In [ ]:
model.layers[7].trainable = False
model.layers[4].trainable = False
model.layers[2].trainable = False
model.layers[0].trainable = False

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

### Train model on single stock

In [ ]:
tensorboard = TensorBoard(log_dir='logs/{}_2'.format(time_stamp))

In [ ]:
history2 = model.fit(x_train, y_train, epochs=45, validation_data=(x_valid, y_valid), batch_size=64, shuffle=False, callbacks=[tensorboard])

### Visualize results

In [ ]:
plot_loss(history1)

In [ ]:
plot_loss(history2)

In [ ]:
print("Training RMSE = {}".format(evaluate_model_rmse(model.predict(x_train), 
                                                      y_train,
                                                     num_features,
                                                     single_stock_scaler_models[target_stock])))

In [ ]:
print("Validation RMSE = {}".format(evaluate_model_rmse(model.predict(x_valid), 
                                                      y_valid,
                                                     num_features,
                                                     single_stock_scaler_models[target_stock])))

In [ ]:
print("Testing RMSE = {}".format(evaluate_model_rmse(model.predict(x_test), 
                                                     y_test,
                                                    num_features,
                                                    single_stock_scaler_models[target_stock])))

#### Compare to simple baseline

In [ ]:
def predict_no_change(x):
    return x[-1][0]

In [ ]:
no_change_preds = np.array([predict_no_change(x) for x in x_train])
no_change_preds = np.expand_dims(no_change_preds, axis=1)

In [ ]:
evaluate_model_rmse(no_change_preds, y_train, num_features, single_stock_scaler_models['XOM'])

In [ ]:
no_change_preds = np.array([predict_no_change(x) for x in x_test])
no_change_preds = np.expand_dims(no_change_preds, axis=1)

In [ ]:
evaluate_model_rmse(no_change_preds, y_test, num_features, single_stock_scaler_models['XOM'])

### Plot data against predictions

In [ ]:
price_vs_preds = pd.DataFrame([x_train[:, -1, 0], model.predict(x_train)[:, 0]]).T
price_vs_preds.columns = ['Price', 'Prediction'] 
price_vs_preds.plot(figsize=(12,8), title="Training performance")

In [ ]:
price_vs_preds = pd.DataFrame([x_valid[:, -1, 0], model.predict(x_valid)[:, 0]]).T
price_vs_preds.columns = ['Price', 'Prediction']
price_vs_preds.plot(figsize=(12,8), title="Validation performance")

In [ ]:
price_vs_preds = pd.DataFrame([x_test[:, -1, 0], model.predict(x_test)[:, 0]]).T
price_vs_preds.columns = ['Price', 'Prediction']
price_vs_preds.plot(figsize=(12,8), title="Testing performance")

In [ ]:
from sklearn.decomposition import PCA

pca = PCA().fit(x_test[0])

print(pca.explained_variance_ratio_.cumsum())

plt.plot(pca.explained_variance_ratio_.cumsum(), lw=3, color='#087E8B')
plt.title('Cumulative explained variance by number of principal components')
plt.show()

In [ ]:
loadings = pd.DataFrame(
    data=pca.components_.T * np.sqrt(pca.explained_variance_), 
    columns=[str(features[i]) for i in range(0, len(x_train[0][0]))],
    index=x_train[0][0]
)
loadings

#print(max(loadings.head()))



In [ ]:
print(max(loadings.head()))

In [ ]:
import shap
i = 0

for i in range(96):
    print(str(top100tech[i]) + " vs " + "Company of Interest (XOM)")
    shap_values = x_valid[i]
    shap.summary_plot(shap_values, x_test[0], features)
    i += 1
    #reporting only 20 out of 29 features
"""
shap_values = x_train
shap.summary_plot(shap_values, x_test[0], features)

""""""
shap_values = x_train[1]
shap.summary_plot(shap_values, x_test[0], features)"""

In [ ]:
#Shap deepexplainer implementation
#deepexplainer and #gradientExplainer need to see python and tensorflow downgrades for potential results
"""
shap.initjs()
explainer = shap.DeepExplainer(model,  x_train)

shap_values = explainer.shap_values(x_test)
"""

In [ ]:
!{sys.executable} -m pip install --user eli5
import eli5
from eli5.sklearn import PermutationImportance

eli5.show_weights(history2, feature_names =features)

In [ ]:
!{sys.executable} -m pip install --user lime
import lime
import lime.lime_tabular
from __future__ import print_function
np.random.seed(1)
#class_names=iris.target_names, third param
explainer = lime.lime_tabular.LimeTabularExplainer(x_train[0], feature_names=features, class_names=features, discretize_continuous=True)

i = np.random.randint(0, x_test.shape[0])
exp = explainer.explain_instance(x_test[i], model.predict_proba, num_features=2, top_labels=1)

exp.show_in_notebook(show_table=True, show_all=False)

In [ ]:
"""Transfer LSTM

Standard Features
Training RMSE[341] = 1.636489156515808
Validation RMSE[342] = 2.2414234268439777
Testing RMSE[343] = 2.0931080358854017
evaluate_model_rmse[356]  = 1.6716020355551713
evaluate_model_rmse[358]  = 1.9969493665689286
PCA Featureset
Training RMSE[341] =  1.615879552138441
Validation RMSE[342] = 2.2138809624137132
Testing RMSE[343] = 2.1154351311017634
evaluate_model_rmse[356]  = 1.6716020355551713
evaluate_model_rmse[358]  = 1.9969493665689286
Pearson, Spearman and Kendall Featureset
Training RMSE[341] = 1.5830068805142028
Validation RMSE[342] = 2.0220939609157274
Testing RMSE[343] = 2.164431030408977
evaluate_model_rmse[356]  = 1.6445849337931098
evaluate_model_rmse[358]  = 1.9969493665689286
PCA Intersection Featureset
Training RMSE[341] = 1.585805755738383
Validation RMSE[342] = 2.2756481983664374
Testing RMSE[343] = 2.112961316220773
evaluate_model_rmse[356]  = 1.6716020355551713
evaluate_model_rmse[358]  = 1.9969493665689286"""